In [1]:
from google.colab import drive

# Monter Google Drive dans l’environnement Colab
drive.mount('/content/drive')



Mounted at /content/drive


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

df1=pd.read_csv('/content/drive/MyDrive/task2 (1)/delivery_with_feature.csv')
df2=pd.read_csv('/content/drive/MyDrive/task2 (1)/pickup_with_new_features.csv')
df1.head()
df2.head()


,region_id,city,accept_time,time_window_start,time_window_end,lng,lat,aoi_id,aoi_type,pickup_time,pickup_gps_time,pickup_gps_lng,pickup_gps_lat,accept_gps_time,accept_gps_lng,accept_gps_lat,ds,waiting_time_minutes,ETA_pickup,city_encoded
0,0,3,2023-07-08 08:13:00,2023-07-08 09:00:00,2023-07-08 11:00:00,121.52223,30.90747,46,14,2023-07-08 10:10:00,2023-07-08 10:10:00,121.52208,30.90836,NaN,118.541358,32.703762,708,117.0,117.0,3
1,0,3,2023-07-21 08:14:00,2023-07-21 09:00:00,2023-07-21 11:00:00,121.52223,30.90742,46,14,2023-07-21 10:10:00,2023-07-21 10:10:00,121.52689,30.91897,2023-07-21 08:14:00,121.503340,30.904240,721,116.0,116.0,3
2,0,3,2023-07-12 07:40:00,2023-07-12 17:00:00,2023-07-12 19:00:00,121.52229,30.90731,46,14,2023-07-12 17:22:00,2023-07-12 17:22:00,121.52612,30.91764,2023-07-12 07:37:00,121.497390,30.906950,712,582.0,582.0,3
3,0,3,2023-07-09 15:38:00,2023-07-09 17:00:00,2023-07-09 19:00:00,121.52234,30.90749,46,14,2023-07-09 15:54:00,2023-07-09 15:54:00,121.52316,30.90876,2023-07-09 15:38:00,121.518140,30.906120,709,16.0,16.0,3
4,0,3,2023-07-07 07:25:00,2023-07-07 09:00:00,2023-07-07 11:00:00,121.52230,30.90747,46,14,2023-07-07 09:53:00,2023-07-07 09:53:00,121.51871,30.90687,2023-07-07 07:25:00,121.497360,30.907520,707,148.0,148.0,3


In [7]:
print(df1.columns)


Index(['order_id', 'region_id', 'courier_id', 'lng', 'lat', 'aoi_id',
       'accept_time', 'accept_gps_time', 'accept_gps_lng', 'accept_gps_lat',
       'delivery_time', 'delivery_gps_time', 'delivery_gps_lng',
       'delivery_gps_lat', 'ds', 'delivery_time_minutes', 'accept_hour',
       'accept_day', 'ETA_delivery', 'city_encoded', 'accept_time_seconds',
       'delivery_time_seconds', 'accept_gps_time_seconds',
       'delivery_gps_time_seconds', 'task_duration', 'distance', 'date',
       'day_of_week', 'hour_of_day', 'task_duration_seconds',
       'log_task_duration'],
      dtype='object')


In [8]:
print(df2.columns)


Index(['region_id', 'city', 'accept_time', 'time_window_start',
       'time_window_end', 'lng', 'lat', 'aoi_id', 'aoi_type', 'pickup_time',
       'pickup_gps_time', 'pickup_gps_lng', 'pickup_gps_lat',
       'accept_gps_time', 'accept_gps_lng', 'accept_gps_lat', 'ds',
       'waiting_time_minutes', 'ETA_pickup', 'city_encoded'],
      dtype='object')


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

cols_to_drop = ['accept_time', 'region_id', 'city', 'ds', 'aoi_type']
cols_to_drop_existing = [col for col in cols_to_drop if col in df1.columns]

df = df1.drop(columns=cols_to_drop_existing)
# === 2. Conversion des dates ===
date_cols = ['accept_time', 'accept_gps_time', 'delivery_time', 'delivery_gps_time', 'date']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# === 3. Convertir task_duration (hh:mm:ss) en secondes ===
if 'task_duration' in df.columns:
    df['task_duration_seconds_conv'] = pd.to_timedelta(df['task_duration'], errors='coerce').dt.total_seconds()

# === 4. Extraction de features temporelles ===
if 'delivery_time' in df.columns:
    df['delivery_hour'] = df['delivery_time'].dt.hour
    df['delivery_weekday'] = df['delivery_time'].dt.dayofweek

if 'accept_time' in df.columns:
    df['accept_hour'] = df['accept_time'].dt.hour
    df['accept_weekday'] = df['accept_time'].dt.dayofweek


# === 6. Gestion des valeurs manquantes ===
df = df.fillna(df.mean(numeric_only=True))

# === 7. Séparation X / y ===
X = df.drop(columns=['ETA_delivery'])  # ou 'ETA_pickup' selon ton besoin
y = df['ETA_delivery']

# Garder uniquement les colonnes numériques
X = X.select_dtypes(include=[np.number])

# === 8. Séparation train/test ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === 9. Lasso Regression ===
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

print("📊 Lasso Regression Evaluation:")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lasso)))
print("MAE :", mean_absolute_error(y_test, y_pred_lasso))
print("R²  :", r2_score(y_test, y_pred_lasso))
print("-" * 40)



📊 Lasso Regression Evaluation:
RMSE: 1.6542329808839125e-06
MAE : 5.231347229676923e-07
R²  : 1.0
----------------------------------------


In [3]:

# === 10. Random Forest Regressor ===
rf = RandomForestRegressor(n_estimators=20, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("📊 Random Forest Regressor Evaluation:")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("MAE :", mean_absolute_error(y_test, y_pred_rf))
print("R²  :", r2_score(y_test, y_pred_rf))
print("-" * 40)

📊 Random Forest Regressor Evaluation:
RMSE: 34.2270607099154
MAE : 0.10047418140440445
R²  : 0.9967427664734959
----------------------------------------


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === 1. Chargement et nettoyage ===
cols_to_drop = ['accept_time', 'region_id', 'city', 'ds', 'aoi_type']
cols_to_drop_existing = [col for col in cols_to_drop if col in df2.columns]
df = df2.drop(columns=cols_to_drop_existing)

# === 2. Conversion des dates ===
date_cols = ['accept_time', 'accept_gps_time', 'pickup_time', 'pickup_gps_time']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')


# === 4. Extraction de features temporelles ===
if 'pickup_time' in df.columns:
    df['pickup_hour'] = df['pickup_time'].dt.hour
    df['pickup_weekday'] = df['pickup_time'].dt.dayofweek

if 'accept_time' in df.columns:
    df['accept_hour'] = df['accept_time'].dt.hour
    df['accept_weekday'] = df['accept_time'].dt.dayofweek

# === 5. Gestion des valeurs manquantes ===
df = df.fillna(df.mean(numeric_only=True))

# === 6. Séparation X / y ===
X = df.drop(columns=['ETA_pickup'])
y = df['ETA_pickup']
X = X.select_dtypes(include=[np.number])

# === 7. Split données ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# === Lasso Regression ===
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

print("📊 Lasso Regression Evaluation:")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lasso)))
print("MAE :", mean_absolute_error(y_test, y_pred_lasso))
print("R²  :", r2_score(y_test, y_pred_lasso))
print("-" * 40)


📊 Lasso Regression Evaluation:
RMSE: 0.00024057736089809367
MAE : 0.00011138039061037112
R²  : 0.9999999999996722
----------------------------------------


In [6]:
# === Random Forest Regressor ===
rf = RandomForestRegressor(n_estimators=20, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("📊 Random Forest Regressor Evaluation:")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("MAE :", mean_absolute_error(y_test, y_pred_rf))
print("R²  :", r2_score(y_test, y_pred_rf))
print("-" * 40)


📊 Random Forest Regressor Evaluation:
RMSE: 9.652584457559286
MAE : 0.017759547925001926
R²  : 0.9994722292204282
----------------------------------------
